## **Chapter V. Task**

We will continue our training with a problem from Kaggle.com. 
In this chapter, we will implement all the validation schemes, some hyperparameter tuning methods, and feature selection methods described above. Measure quality metrics on training and test samples. Will detect overfitted models and regularize them. And dive deeper with native model estimation and comparison.

### 1. **Answer the questions from the introduction**

#### a. What is leave-one-out? Provide limitations and strengths.  

Leave-One-Out кросс валидация - это частный случай k-fold Cross Validation, где k = n. Данные размерности 10 делятся на обучающую и тестовую выборки по принципу 9 обучающей и 1 тестовая на каждой итерации для всех данных. При этом,каждый набор данных __ровно один раз__ становится тестовым (см. пример кода.). Другими словами, для **_n_** образцов мы имеем **_n-1_** тренировочных данных и **1** тестовый набор.
Достоинства:
 * максимальное использование данных для обучения
 * не зависит от случайного разбиения
 * эффективен только на маленьких наборах данных

Недостатки:
 * высокая стоимость вычислительных операций для больших выборок данных - O(n)
 * высокая дисперсия оценки
 * может недооценивать ошибку на новых данных
 * имеет нестабильную оценку

```Python
>>> from sklearn.model_selection import LeaveOneOut

>>> X = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
>>> loo = LeaveOneOut()
>>> for train, test in loo.split(X):
...     print("%s %s" % (train, test))

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]
```

Источник: https://scikit-learn.org/stable/modules/cross_validation.html#leave-one-out

#### b. How do Grid Search, Randomized Grid Search, and Bayesian optimization work?  

In [1]:
A = {1, 2, 3, 4, 5}
B = {6, 7, 8, 9}
C = {10, 11, 12}

def cartesian_product(*sets):
    """
    Декартово произведение множеств
    """
    result = [[]]
    for pool in sets:
        result = [x + [y] for x in result for y in pool]
    return [tuple(item) for item in result]

decart = cartesian_product(A, B, C)

> Этапы работы поиска наилучших гиперпараметров через Grid Search:
 1. **Определение пространства внешних гиперпараметров**. На первом этапе необходимо обозначить какие гиперпараметры мы ищем. Параметр должен содержать множество значений, по которым мы будем проводить поиск. Параметров может быть несколько. Несколько параметров собираются в единый словарь, где ключи — имена гиперпараметров, а значения — списки возможных значений для каждого параметра.
 2. **Создание сетки гиперпараметров**. Создается декартово произведение всех списков значений, получая полный набор всех возможных комбинаций гиперпараметров.
 3. **Проход по списку гиперпараметров**. Алгоритм GridSearch последовательно проходит по каждой комбинации гиперпараметров, обучается и оценивается с помощью k-fold CV, вычисляется средняя оценка гиперпараметров по всем фолдам для заданной метрики.
 4. **Сохранение и возвращение результата**. По окончании подбора параметров выбирается комбинация с наилучшей средней оценкой. GridSearch возвращает: лучшие гиперпараметры `best_params_`; лучшие оценки `best_score_`; полную информацию по всем комбинациям `cv_results_`.

> Randomized Grid Search:
 * Принцип работы Randomized Grid Search схож с работой Grid Search, но не создает полную сетку всех комбинаций гиперпараметров. Вместо этого поиск происходит случайным образом. Еще одно отличие - это наличие итераций. На каждой итерации `n_iter` случайным образом генерируется комбинация гиперпараметров, происходит обучение модели и оценка её качества с помощью k-fold CV на всех фолдах. Случайный выбор позволяет ускорить процесс подбора гиперпараметров. В таком случае, гиперпараметры следует задавать не только дискретным образом, но и в диапазоне и\или распределении, например: `stats.uniform(0.1, 10)`, здесь будет создано распределение случайных чисел от 0.1 до 10 + 0.1. Случайное число из данного распределения будет получено внутри GridSearch.

> Bayesian optimization
 * Принцип работы Bayesian optimization отличается от Grid Search и Randomized Search тем, что новые комбинации гиперпараметров выбираются не случайно и не полным перебором, а “осмысленно” на основе результатов уже проведённых экспериментов.

 * На каждом шаге строится вероятностная аппроксимация (surrogate model) функции качества: как метрика зависит от гиперпараметров. Чаще всего в роли surrogate используется Gaussian Process, но также могут применяться, например, TPE или другие модели.

 * Далее рассчитывается acquisition function — функция, которая определяет, какую точку в пространстве гиперпараметров выгоднее проверить следующей. Она балансирует два режима:
   * exploitation — проверять области, где модель ожидает высокий скор,
   * exploration — проверять области, где неопределённость высокая (мало информации), чтобы не пропустить лучший максимум.
 * Примеры acquisition: Expected Improvement (EI), Probability of Improvement (PI), Upper Confidence Bound (UCB).

 * После выбора очередной комбинации гиперпараметров модель обучается и оценивается (обычно через k-fold CV), полученная метрика добавляется в “историю”, и surrogate-модель обновляется.

 * Процесс повторяется до достижения лимита по итерациям/времени. В итоге возвращается лучшая найденная комбинация гиперпараметров и соответствующая оценка качества.

#### c. Explain classification of feature selection methods. Explain how Pearson and Chi2 work. Explain how Lasso works. Explain what permutation significance is. Become familiar with SHAP.  

__1. Классификация методов отбора признаков (feature selection):__
 * Методы отбора признаков обычно делят на 3 группы:
    1. Filter methods (фильтры) — отбор признаков до обучения модели, на основе статистик данных. Быстрые и простые, но не учитывают конкретную модель. Примеры: корреляция (Pearson), Chi2, mutual information, ANOVA.
    2. Wrapper methods (обёртки) — отбор признаков через многократное обучение модели и сравнение качества на разных наборах фич. Обычно дают хороший результат, но дорогие по времени. Примеры: RFE (recursive feature elimination), forward/backward selection.
    3. Embedded methods (встроенные) — отбор происходит внутри обучения модели, то есть модель сама “зануляет/ослабляет” ненужные признаки. Примеры: Lasso (L1), деревья/градиентный бустинг (importance по сплитам).

__2. Как работает Pearson (корреляция Пирсона):__
   * Pearson измеряет линейную связь между двумя числовыми переменными (например, фичей X и таргетом y).
   * Значение корреляции лежит в диапазоне [-1, 1]:
      * +1 — идеальная прямая линейная связь,
      * -1 — идеальная обратная линейная связь,
      * 0 — нет линейной зависимости (но может быть нелинейная).
   * Для feature selection часто берут |corr(X, y)| и выбирают top-k признаков: чем больше абсолютная корреляция, тем сильнее линейная связь с таргетом.

__3. Как работает Chi2 (хи-квадрат) для отбора признаков:__

   * Chi2 — статистический критерий, который проверяет зависимость между признаком и классом.
   * Обычно применяется для классификации и для признаков, которые можно интерпретировать как частоты/неотрицательные значения (например, counts в Bag-of-Words).
   * Идея: строится таблица “наблюдаемое vs ожидаемое”. Если признак независим от класса, распределение значений признака по классам должно быть примерно таким же, как ожидается при независимости. Чем больше расхождение — тем больше Chi2-статистика и тем более “полезен” признак.
   * На практике в sklearn: считают Chi2-скор для каждой фичи, затем выбирают top-k по величине.

__4. Как работает Lasso (L1-регуляризация) как отбор признаков:__

   * Lasso — это линейная модель, которая добавляет к функции потерь штраф L1 на веса:
      * модель старается минимизировать ошибку + alpha * сумма(|w_i|).
   * Особенность L1: она “толкает” часть весов ровно в 0, то есть признаки с нулевыми весами можно считать отброшенными.
   * Поэтому Lasso — это embedded feature selection: отбор делается автоматически во время обучения.
   * Чем больше alpha, тем сильнее регуляризация и тем больше признаков будет занулено (но можно переужесточить и потерять качество).

__5. Что такое permutation significance / permutation importance:__

   * Permutation importance измеряет важность признака через падение качества модели при его перемешивании.
   * Алгоритм:
      * Считаем базовую метрику на валидации/тесте.
      * Берём один признак и перемешиваем его значения между объектами (разрываем связь с таргетом).
      * Снова считаем метрику. Если качество сильно ухудшилось — признак важный.
      *Повторяем несколько раз (n_repeats) и усредняем.
   * Важность обычно интерпретируется как разница метрики: metric_permuted - metric_base (или аналогично со “score” у sklearn).
   * Метод модель-агностичный (подходит почти для любой модели), но может быть дорогим по времени.

__6. SHAP (введение, что это и зачем):__

   * SHAP (SHapley Additive exPlanations) — метод интерпретации моделей, основанный на идее Shapley values из теории игр.
   * Он раскладывает предсказание модели на вклад отдельных признаков:
      * есть базовое значение (expected value),
      * и есть вклад каждой фичи, которые в сумме дают предсказание модели.
   * SHAP можно использовать для:
      * анализа важности признаков (среднее |shap| по датасету),
      * локальных объяснений (почему модель так предсказала для конкретного объекта),
      * поиска неожиданных зависимостей/ошибок в данных.
   * Для разных моделей есть разные explainers:
      * `TreeExplainer` — для деревьев/бустинга (быстро),
      * `LinearExplainer` — для линейных моделей,
      * `KernelExplainer` — универсальный, но самый медленный.

### **2. Introduction — do all the preprocessing from the previous lesson**

In [2]:
# 2.Imports
import pandas as pd
import numpy as np
import re
from math import ceil

#### a. Read all the data.

In [3]:
# 2.a
df = pd.read_json("./datasets/train.json", convert_dates=["created"])
df.head(3)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium


#### b. Create features: 'Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed', 'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding', 'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck', 'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony', 'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace'.  

In [4]:
# 2.b
pattern = re.compile(r"[\-\[\]'\"\&]")
remove_sym = (lambda x: list([re.sub(pattern, " ", elem.title()) for elem in x]))
remove_ws = (lambda x: list([elem.replace(" ", "") for elem in x]))

df_prepare = df.copy()

df_prepare["features"] = (
    df_prepare["features"]
    .apply(remove_sym)  # удаляем лишнее
    .apply(remove_ws)  # удаляем пробелы
)
target = ["price"]
rooms = ["bedrooms", "bathrooms"]
tags = ['Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed',
            'Doorman', 'Dishwasher', 'NoFee', 'LaundryInBuilding',
            'FitnessCenter', 'PreWar', 'LaundryInUnit', 'RoofDeck',
            'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony',
            'SwimmingPool', 'NewConstruction', 'Terrace',]

for tag in tags:
    df_prepare[tag] = (
        df_prepare["features"]
        .apply(lambda x: 1 if tag in x else 0)
    )

df_prepare = df_prepare[["building_id", "created"] + rooms + tags + target]
df_prepare = df_prepare[
    df_prepare[target[0]].between(
        df_prepare[target[0]].quantile(0.01), # 1475
        df_prepare[target[0]].quantile(0.99), # 13000
        inclusive="both"
    )
].reset_index(drop=True)
df_prepare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48379 entries, 0 to 48378
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   building_id        48379 non-null  object        
 1   created            48379 non-null  datetime64[ns]
 2   bedrooms           48379 non-null  int64         
 3   bathrooms          48379 non-null  float64       
 4   Elevator           48379 non-null  int64         
 5   HardwoodFloors     48379 non-null  int64         
 6   CatsAllowed        48379 non-null  int64         
 7   DogsAllowed        48379 non-null  int64         
 8   Doorman            48379 non-null  int64         
 9   Dishwasher         48379 non-null  int64         
 10  NoFee              48379 non-null  int64         
 11  LaundryInBuilding  48379 non-null  int64         
 12  FitnessCenter      48379 non-null  int64         
 13  PreWar             48379 non-null  int64         
 14  Laundr

### 3. **Implement the next methods:**

#### a. Split data into 2 parts randomly with parameter test_size (ratio from 0 to 1), return training and test samples.

In [ ]:
# 3.a
def my_train_test_split(X, y, test_size=0.20, shuffle=True, random_state=None):
    if not (X.index.equals(y.index)):
        raise ValueError("X и y должны быть элементами одного массива")
    if not (0 < test_size < 1):
        raise ValueError("`test_size` должен быть 0 < `test_size` < 1")

    n = len(X)
    n_test = ceil(n * test_size)

    if shuffle:
        rdm = np.random.RandomState(random_state)
        idxs = rdm.permutation(np.arange(n))
    else:
        idxs = np.arange(n)

    test_idxs = idxs[:n_test]
    train_idxs = idxs[n_test:]

    return [X.iloc[train_idxs], X.iloc[test_idxs],
            y.iloc[train_idxs], y.iloc[test_idxs]]


X_train, X_test, y_train, y_test = my_train_test_split(
    df_prepare.drop(columns=target),
    df_prepare["price"],
    test_size=0.2,
    random_state=21,
)
print(
    "Размерности совпадают: " +
    f"{len(X_train) + len(X_test) == len(df_prepare)}",
    "\nРазмерности: " +
    f"train{X_train.shape}, test{X_test.shape}",
    "\nПересечения отсутствуют:", all([
        X_train.index.intersection(X_test.index).empty,
        y_train.index.intersection(y_test.index).empty
    ])
)

Размерности совпадают: True 
Размерности: train(38703, 23), test(9676, 23) 
Пересечения отсутствуют: True


#### b. Randomly split data into 3 parts with parameters validation_size and test_size, return train, validation and test samples.

In [6]:
# 3.b
def my_train_valid_test_split(X,
                              y,
                              valid_size=0.25,
                              test_size=0.20,
                              shuffle=True,
                              random_state=None):
    if not (X.index.equals(y.index)):
        raise ValueError("X и y должны быть элементами одного массива")
    if not (0 < test_size < 1):
        raise ValueError("`test_size` должен быть 0 < `test_size` < 1")
    if not (0 < valid_size < 1):
        raise ValueError("`test_size` должен быть 0 < `valid_size` < 1")
    if not (0 < valid_size + test_size < 1):
        raise ValueError("`test_size` должен быть 0 < `test_size` + `valid_size` < 1")

    n = len(X)
    n_test = ceil(n * test_size)
    n_valid = ceil((n - n_test) * valid_size)
    idxs = np.arange(n)

    if shuffle:
        rdm = np.random.RandomState(random_state)
        idxs = rdm.permutation(idxs)

    test_idx = idxs[:n_test]
    valid_idx = idxs[n_test: n_valid + n_test]
    train_idx = idxs[n_valid + n_test:]

    return [X.iloc[train_idx], X.iloc[valid_idx], X.iloc[test_idx],
            y.iloc[train_idx], y.iloc[valid_idx], y.iloc[test_idx]]


X_train, X_valid, X_test, y_train, y_valid, y_test = my_train_valid_test_split(
    df_prepare.drop(columns=target),
    df_prepare["price"],
    valid_size=0.25,
    test_size=0.2,
    random_state=21,
)
print(
    "Размерности совпадают: " +
    f"{len(X_train) + len(X_valid) + len(X_test) == len(df_prepare)}",
    "\nРазмерности: " +
    f"train: {X_train.shape}, valid: {X_valid.shape}, test: {X_test.shape}",
    "\nПересечения отсутствуют:", all([
        X_train.index.intersection(X_test.index).empty,
        X_train.index.intersection(X_valid.index).empty,
        X_test.index.intersection(X_valid.index).empty,
    ])
)

Размерности совпадают: True 
Размерности: train: (29027, 23), valid: (9676, 23), test: (9676, 23) 
Пересечения отсутствуют: True


#### c. Split data into 2 parts with parameter date_split, return train and test samples split by date_split param.

In [7]:
# 3.c
def my_date_train_test_split(X, y, date_col, date_split=0.20):
    if not (X.index.equals(y.index)):
        raise ValueError("X и y должны быть элементами одного массива")
    if not (X.index.equals(date_col.index)):
        raise ValueError("X и `date_col` должны быть элементами одного массива")
    if not (0 < date_split < 1):
        raise ValueError("`date_split` должен быть 0 < `date_split` < 1")

    n_test = ceil(len(X) * date_split)
    sort_dates = date_col.sort_values().index[::-1]

    test_idx = sort_dates[:n_test]
    train_idx = sort_dates[n_test:]

    return [X.loc[train_idx], X.loc[test_idx],
            y.loc[train_idx], y.loc[test_idx]]


X_train, X_test, y_train, y_test = my_date_train_test_split(
    df_prepare.drop(columns=target),
    df_prepare["price"],
    date_col=df_prepare["created"],
    date_split=0.20,
)
print(
    "Размерности совпадают: " +
    f"{len(X_train) + len(X_test) == len(df_prepare)}",
    "\nРазмерности: " +
    f"train: {X_train.shape}, test: {X_test.shape}",
    "\nПересечения отсутствуют:", all([
        X_train.index.intersection(X_test.index).empty,
        y_test.index.intersection(y_train.index).empty,
    ]),
    end="\n" * 2
)
print(
    f'`X_train` start: {X_train["created"].min()}',
    f'`X_train` end:   {X_train["created"].max()}',
    f'`X_test`  start: {X_test["created"].min()}',
    f'`X_test`  end:   {X_test["created"].max()}',
    sep="\n"
)

Размерности совпадают: True 
Размерности: train: (38703, 23), test: (9676, 23) 
Пересечения отсутствуют: True

`X_train` start: 2016-04-01 22:12:41
`X_train` end:   2016-06-12 08:07:17
`X_test`  start: 2016-06-12 08:07:50
`X_test`  end:   2016-06-29 21:41:47


#### d. Split data into 3 parts with parameters validation_date and test_date, return train, validation and test samples split by input params.

In [ ]:
# 3.d
def my_date_train_valid_test_split(X, y, date_col, valid_date=0.20, test_date=0.20):
    if not (X.index.equals(y.index)):
        raise ValueError("X и y должны быть элементами одного массива")
    if not (0 < test_date < 1):
        raise ValueError("`test_date` должен быть 0 < `test_date` < 1")
    if not (0 < valid_date < 1):
        raise ValueError("`test_date` должен быть 0 < `valid_date` < 1")
    if not (0 < valid_date + test_date < 1):
        raise ValueError("`test_date` должен быть 0 < `test_date` + `valid_date` < 1")

    n = len(X)
    n_test = ceil(n * test_date)
    n_valid = ceil((n - n_test) * valid_date)
    sort_dates = date_col.sort_values().index[::-1]

    test_idxs = sort_dates[:n_test]
    valid_idxs = sort_dates[n_test: n_valid + n_test]
    train_idxs = sort_dates[n_valid + n_test:]

    return [X.loc[train_idxs], X.loc[valid_idxs], X.loc[test_idxs],
            y.loc[train_idxs], y.loc[valid_idxs], y.loc[test_idxs]]


X_train, X_valid, X_test, y_train, y_valid, y_test = my_date_train_valid_test_split(
    df_prepare.drop(columns=target),
    df_prepare["price"],
    date_col=df_prepare["created"],
    valid_date=0.25,
    test_date=0.20,
)
print(
    "Размерности совпадают: " +
    f"{len(X_train) + len(X_valid) + len(X_test) == len(df_prepare)}",
    "\nРазмерности: " +
    f"train: {X_train.shape}, valid: {X_valid.shape}, test: {X_test.shape}",
    "\nПересечения отсутствуют:", all([
        X_train.index.intersection(X_test.index).empty,
        X_train.index.intersection(X_valid.index).empty,
        X_test.index.intersection(X_valid.index).empty,
    ]),
    end="\n" * 2
)
print(
    f'`X_train` start: {X_train["created"].min()}',
    f'`X_train` end:   {X_train["created"].max()}',
    f'`X_valid` start: {X_valid["created"].min()}',
    f'`X_valid` end:   {X_valid["created"].max()}',
    f'`X_test`  start: {X_test["created"].min()}',
    f'`X_test`  end:   {X_test["created"].max()}',
    sep="\n"
)

Размерности совпадают: True 
Размерности: train: (29027, 23), valid: (9676, 23), test: (9676, 23) 
Пересечения отсутствуют: True

`X_train` start: 2016-04-01 22:12:41
`X_train` end:   2016-05-24 16:34:40
`X_valid` start: 2016-05-24 16:40:27
`X_valid` end:   2016-06-12 08:07:17
`X_test`  start: 2016-06-12 08:07:50
`X_test`  end:   2016-06-29 21:41:47


#### e. Make split procedure determenistic. What does it mean?

In [9]:
# 3.e
"""
    Каждая реализация my_train_test_split сопровождена детерминацией, однако,
    в функциях со стратегией 'Out-of-time' (например, my_date_train_test_split)
    детерминация не требуется, поскольку данные разделяются по времени,
    а не случайным образом.
""";

### 4. **Implement the next cross-validation methods:**

#### a. K-Fold, where k is the input parameter, returns a list of train and test indices.

In [10]:
# 4.a
def my_kfold_cv(X, k=5):
    n = len(X)
    if k <= 1 or k > n:
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    idxs = np.arange(n)

    folds = np.array_split(idxs, k)
    for test_idx in folds:
        test_idx = np.array(sorted(test_idx))
        train_idx = np.setdiff1d(idxs, test_idx, assume_unique=True)
        yield (train_idx, test_idx)

#### b. Grouped K-Fold, where k and group_field are input parameters, returns list of train and test indices.

In [11]:
# 4.b
def my_group_kfold_cv(X, group_field, k=5):
    n = len(X)
    if k <= 1 or k > n:
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    group_values = group_field.to_numpy()
    idxs = np.arange(n)

    unique_groups, inv_idx, counts = np.unique(
        group_values, return_inverse=True, return_counts=True
    )
    indices_by_group = {}
    for i, g in enumerate(unique_groups):
        indices_by_group[g] = np.where(inv_idx == i)[0].tolist()

    order = np.argsort(counts)[::-1]
    unique_groups, counts = (unique_groups[order], counts[order])

    folds, sizes_folds = ([[] for _ in range(k)], np.zeros(k, dtype=int))
    for g, c in zip(unique_groups, counts):
        i_fold = np.argmin(sizes_folds)
        folds[i_fold].extend(indices_by_group[g])
        sizes_folds[i_fold] += c

    for test_idx in folds:
        test_idx = np.array(sorted(test_idx))
        train_idx = np.setdiff1d(idxs, test_idx, assume_unique=True)
        yield (train_idx, test_idx)

#### c. Stratified K-fold, where k and stratify_field are input parameters, returns list of train and test indices.

In [12]:
df_prepare["price_category"] = pd.cut(
    df_prepare["price"],
    bins=[-np.inf, 2700, 3800, np.inf],
    labels=["low", "middle", "high"]
)
df_prepare["price_category"].value_counts()

price_category
middle    17269
low       16794
high      14316
Name: count, dtype: int64

In [13]:
# 4.c
def my_stratified_kfold_cv(X, stratify_field, k=5):
    n = len(X)
    if k <= 1 or k > n:
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    stratify_values = stratify_field.to_numpy()
    idxs = np.arange(n)

    unique_classes, counts = np.unique(stratify_values, return_counts=True)
    if counts.min() < k:
        raise ValueError("Минимальный класс содержит меньше объектов, чем k")

    folds = [[] for _ in range(k)]
    for cls in unique_classes:
        class_idxs = np.where(stratify_values == cls)[0]
        parts = np.array_split(class_idxs, k)
        for i in range(k):
            folds[i].append(parts[i])

    folds = [np.concatenate(f) for f in folds]
    for test_idx in folds:
        test_idx = np.array(test_idx)
        train_idx = np.setdiff1d(idxs, test_idx, assume_unique=True)
        yield (train_idx, test_idx)

#### d. Time series split, where k and date_field are input parameters, returns list of train and test indices.

In [14]:
# 4.d
def my_time_series_split(X, date_field, k=5):
    n = len(X)
    if k <= 1 or k > n:
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    dates = X[date_field].reset_index(drop=True).index.to_numpy()
    order = np.argsort(dates, kind="mergesort")

    n_folds = k + 1
    test_size = n // n_folds
    remainder = n % n_folds
    train_end = remainder + test_size

    for _ in range(k):
        test_start = train_end
        test_end = test_start + test_size
        yield (order[:train_end], order[test_start: test_end])
        train_end = test_end

### 5. **Cross-validation comparison**

#### a. Apply all the validation methods implemented above to our dataset. To apply Stratified algorithm you should preprocess target.

In [15]:
# 5.a
n_folds = 5

my_train_test_kf = list(my_kfold_cv(df_prepare, k=n_folds))
my_train_test_gkf = list(
    my_group_kfold_cv(df_prepare, df_prepare["building_id"], k=n_folds)
)
my_train_test_skf = list(
    my_stratified_kfold_cv(df_prepare, df_prepare["price_category"], k=n_folds)
)
my_train_test_tss = list(
    my_time_series_split(df_prepare, "created", n_folds)
)

my_methods = {
    "MyKFold": my_train_test_kf,
    "MyGroupKFold": my_train_test_gkf,
    "MyStratifiedKFold": my_train_test_skf,
    "MyTimeSeriesSplit": my_train_test_tss
}

#### b. Apply the appropriate methods from sklearn.

In [16]:
# 5.b
from sklearn.model_selection import (KFold,
                                     GroupKFold,
                                     StratifiedKFold,
                                     TimeSeriesSplit)

train_test_kf = list(KFold(n_folds).split(df_prepare))
train_test_gkf = list(
    GroupKFold(n_folds).split(df_prepare, groups=df_prepare["building_id"])
)
train_test_skf = list(
    StratifiedKFold(n_folds).split(df_prepare, df_prepare["price_category"])
)
train_test_tss = list(
    TimeSeriesSplit(n_folds).split(df_prepare.sort_values(by="created"))
)

base_methods = {
    "KFold": list(train_test_kf),
    "GroupKFold": list(train_test_gkf),
    "StratifiedKFold": list(train_test_skf),
    "TimeSeriesSplit": list(train_test_tss)
}

#### c. Compare the resulting feature distributions for the training part of the dataset between sklearn and your implementation.

In [17]:
# 5.c
compare = (lambda x, y: x.sort_index().equals(y.sort_index()))
for my, base in zip(my_methods.items(), base_methods.items()):
    compare_result = []

    for (tr1, te1), (tr2, te2) in zip(my[1], base[1]):
        compare_result.append(compare(df_prepare.iloc[tr1], df_prepare.iloc[tr2]))
        compare_result.append(compare(df_prepare.iloc[te1], df_prepare.iloc[te2]))
    print(f"Сравнение {my[0]} и {base[0]}:", ("Идентичны"
                                            if all(compare_result)
                                            else "Различаются"))
    print()

Сравнение MyKFold и KFold: Идентичны

Сравнение MyGroupKFold и GroupKFold: Идентичны

Сравнение MyStratifiedKFold и StratifiedKFold: Различаются

Сравнение MyTimeSeriesSplit и TimeSeriesSplit: Идентичны



In [18]:
# 5.c.a
display(
    my_train_test_skf,
    train_test_skf
)

[(array([ 9593,  9594,  9598, ..., 48376, 48377, 48378], shape=(38702,)),
  array([   5,    6,    9, ..., 9615, 9617, 9619], shape=(9677,))),
 (array([    0,     1,     2, ..., 48376, 48377, 48378], shape=(38703,)),
  array([ 9593,  9594,  9598, ..., 19262, 19263, 19264], shape=(9676,))),
 (array([    0,     1,     2, ..., 48376, 48377, 48378], shape=(38703,)),
  array([19148, 19150, 19154, ..., 29093, 29096, 29098], shape=(9676,))),
 (array([    0,     1,     2, ..., 48376, 48377, 48378], shape=(38703,)),
  array([28613, 28618, 28619, ..., 38764, 38766, 38769], shape=(9676,))),
 (array([    0,     1,     2, ..., 38952, 38956, 38957], shape=(38705,)),
  array([38329, 38330, 38331, ..., 48371, 48372, 48374], shape=(9674,)))]

[(array([ 9592,  9593,  9594, ..., 48376, 48377, 48378], shape=(38703,)),
  array([   0,    1,    2, ..., 9815, 9821, 9826], shape=(9676,))),
 (array([    0,     1,     2, ..., 48376, 48377, 48378], shape=(38703,)),
  array([ 9592,  9593,  9594, ..., 19589, 19591, 19593], shape=(9676,))),
 (array([    0,     1,     2, ..., 48376, 48377, 48378], shape=(38703,)),
  array([19144, 19148, 19150, ..., 29300, 29303, 29307], shape=(9676,))),
 (array([    0,     1,     2, ..., 48376, 48377, 48378], shape=(38703,)),
  array([28611, 28613, 28618, ..., 38952, 38956, 38957], shape=(9676,))),
 (array([    0,     1,     2, ..., 38952, 38956, 38957], shape=(38704,)),
  array([38329, 38330, 38331, ..., 48376, 48377, 48378], shape=(9675,)))]

#### d. Compare all validation schemes. Choose the best one. Explain your choice.

##### __5.d__
> Изучив "под капотом" каждый метод кросс валидации, я пришел к выводу, что лучшим методом является `StratifiedKFold` для стратегии `out-of-fold`, т.к. метод `StratifiedKFold` сохраняет баланс в целевых признаках, что уменьшает смещение предсказаний в сторону одной метки. Например, в нашем случае `df["price"]` имеет достаточно высокие __*стандартное отклонение $\left(\sigma ^ 2\right)$ и дисперсию $\left(\sigma = \sqrt{\sigma ^ 2}\right)$*__, что говорит о высоком разбросе целевого признака `"price"` в датафрэйме. Также, визуализация распределения `"price"` из предыдущих задач показала, что `"price"` имеет сильное левостороннее смещение (длинный правый хвост, распределение не является нормальным), а это значит, что модель чаще будет предсказывать значения области "высокой плотности". Мы можем обработать это смещение, либо преобразовать его из чисел в __*диапазоны/категории*__. `StratifiedKFold` требует предобработки числовых меток и это следует учитывать.

### 6. **Feature Selection**

#### a. Fit a Lasso regression model with normalized features. Use your method for splitting samples into 3 parts by field created with 60/20/20 ratio — train/validation/test.

In [19]:
# 6.a
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (mean_absolute_error as MAE,
                             root_mean_squared_error as RMSE,
                             r2_score as R2)

num_features = (df_prepare
                .select_dtypes(include=[np.number])
                .drop(columns=target[0])
                .columns)

X_train, X_valid, X_test, y_train, y_valid, y_test = my_train_valid_test_split(
    df_prepare[num_features],
    df_prepare[target],
    valid_size=0.2,
    test_size=0.2,
    random_state=21,
)

scaler = MinMaxScaler().fit(X_train[num_features])

X_train[num_features] = scaler.transform(X_train[num_features])
X_valid[num_features] = scaler.transform(X_valid[num_features])
X_test[num_features]  = scaler.transform(X_test[num_features])

lasso = Lasso()
lasso.fit(X_train, y_train)

mae, rmse, r2 = (pd.DataFrame({"model": [],
                               "train": [],
                               "valid": [],
                               "test": [],}) for _ in range(3))
mae.loc[len(mae)] = [
    "Lasso MinMaxScaler",
    MAE(y_train, lasso.predict(X_train)),
    MAE(y_valid, lasso.predict(X_valid)),
    MAE(y_test, lasso.predict(X_test))
]
rmse.loc[len(rmse)] = [
    "Lasso MinMaxScaler",
    RMSE(y_train, lasso.predict(X_train)),
    RMSE(y_valid, lasso.predict(X_valid)),
    RMSE(y_test, lasso.predict(X_test))
]
r2.loc[len(r2)] = [
    "Lasso MinMaxScaler",
    R2(y_train, lasso.predict(X_train)),
    R2(y_valid, lasso.predict(X_valid)),
    R2(y_test, lasso.predict(X_test))
]

#### b. Sort features by weight coefficients from model, fit model to top 10 features and compare quality.

In [20]:
# 6.b
feature_names = lasso.feature_names_in_
feature_weights = np.abs(lasso.coef_).round(6)
weights = pd.DataFrame({
    "Name": feature_names,
    "Importance": feature_weights}
)
most_important = weights.sort_values(by="Importance", ascending=False)
display(most_important.head(15))
top10= most_important["Name"][:10].values

lasso = Lasso()
lasso.fit(X_train[top10], y_train)

mae.loc[len(mae)] = [
    "Lasso top10 MinMaxScaler",
    MAE(y_train, lasso.predict(X_train[top10])),
    MAE(y_valid, lasso.predict(X_valid[top10])),
    MAE(y_test, lasso.predict(X_test[top10]))
]
rmse.loc[len(rmse)] = [
    "Lasso top10 MinMaxScaler",
    RMSE(y_train, lasso.predict(X_train[top10])),
    RMSE(y_valid, lasso.predict(X_valid[top10])),
    RMSE(y_test, lasso.predict(X_test[top10]))
]
r2.loc[len(r2)] = [
    "Lasso top10 MinMaxScaler",
    R2(y_train, lasso.predict(X_train[top10])),
    R2(y_valid, lasso.predict(X_valid[top10])),
    R2(y_test, lasso.predict(X_test[top10]))
]

print("MAE")
display(mae)
print("RMSE")
display(rmse)
print("R2")
display(r2)


,Name,Importance
1,bathrooms,15241.470711
0,bedrooms,3242.489612
6,Doorman,643.927529
12,LaundryInUnit,479.440627
9,LaundryInBuilding,265.531995
10,FitnessCenter,250.292821
16,HighSpeedInternet,197.873125
2,Elevator,187.366353
8,NoFee,177.432350
19,NewConstruction,126.675976


MAE


,model,train,valid,test
0,Lasso MinMaxScaler,716.722100,716.634879,711.028730
1,Lasso top10 MinMaxScaler,719.317161,719.755868,714.768131


RMSE


,model,train,valid,test
0,Lasso MinMaxScaler,1039.645459,1045.209477,1028.465037
1,Lasso top10 MinMaxScaler,1043.106897,1048.647653,1032.016266


R2


,model,train,valid,test
0,Lasso MinMaxScaler,0.577598,0.569756,0.583895
1,Lasso top10 MinMaxScaler,0.574780,0.566921,0.581017


#### d. Implement permutation importance method and take top 10 features, refit model and measure quality.

In [21]:
# 6.d
from sklearn.inspection import permutation_importance

lasso = Lasso()
lasso.fit(X_train, y_train)

pi = permutation_importance(
    lasso,
    X_valid,
    y_valid,
    scoring="neg_mean_absolute_percentage_error",
)
imp_mean = pd.Series(pi.importances_mean, index=X_valid.columns).sort_values(ascending=False)
imp_std  = pd.Series(pi.importances_std,  index=X_valid.columns).loc[imp_mean.index]

table = pd.DataFrame({"Mean ± Std Deviation": imp_mean + imp_std})
display(table.head(10))
top10 = imp_mean.head(10).index.tolist()

lasso = Lasso()
lasso.fit(X_train[top10], y_train)

mae.loc[len(mae)] = [
    "Lasso permutation MinMaxScaler",
    MAE(y_train, lasso.predict(X_train[top10])),
    MAE(y_valid, lasso.predict(X_valid[top10])),
    MAE(y_test, lasso.predict(X_test[top10]))
]
rmse.loc[len(rmse)] = [
    "Lasso permutation MinMaxScaler",
    RMSE(y_train, lasso.predict(X_train[top10])),
    RMSE(y_valid, lasso.predict(X_valid[top10])),
    RMSE(y_test, lasso.predict(X_test[top10]))
]
r2.loc[len(r2)] = [
    "Lasso permutation MinMaxScaler",
    R2(y_train, lasso.predict(X_train[top10])),
    R2(y_valid, lasso.predict(X_valid[top10])),
    R2(y_test, lasso.predict(X_test[top10]))
]

print("MAE")
display(mae)
print("RMSE")
display(rmse)
print("R2")
display(r2)


,Mean ± Std Deviation
bathrooms,0.078023
bedrooms,0.076989
Doorman,0.036124
LaundryInUnit,0.011688
FitnessCenter,0.005623
Elevator,0.004625
LaundryInBuilding,0.004018
Dishwasher,0.002861
DogsAllowed,0.001254
HighSpeedInternet,0.000966


MAE


,model,train,valid,test
0,Lasso MinMaxScaler,716.722100,716.634879,711.028730
1,Lasso top10 MinMaxScaler,719.317161,719.755868,714.768131
2,Lasso permutation MinMaxScaler,719.503004,719.809385,714.456090


RMSE


,model,train,valid,test
0,Lasso MinMaxScaler,1039.645459,1045.209477,1028.465037
1,Lasso top10 MinMaxScaler,1043.106897,1048.647653,1032.016266
2,Lasso permutation MinMaxScaler,1045.668346,1051.165187,1032.324159


R2


,model,train,valid,test
0,Lasso MinMaxScaler,0.577598,0.569756,0.583895
1,Lasso top10 MinMaxScaler,0.574780,0.566921,0.581017
2,Lasso permutation MinMaxScaler,0.572689,0.564839,0.580767


#### e. Import Shap and also refit model on top 10 features.

In [22]:
# 6.e
import shap

lasso = Lasso()
lasso.fit(X_train, y_train)

explainer = shap.LinearExplainer(lasso, X_train)
shap_values = explainer.shap_values(X_train)

imp = np.mean(np.abs(shap_values), axis=0)
imp = pd.Series(imp, index=X_train.columns).sort_values(ascending=False)
table = pd.DataFrame({"shap_value": imp})
display(table)
top10 = imp.head(10).index.tolist()

lasso = Lasso()
lasso.fit(X_train[top10], y_train)

mae.loc[len(mae)] = [
    "Lasso shap MinMaxScaler",
    MAE(y_train, lasso.predict(X_train[top10])),
    MAE(y_valid, lasso.predict(X_valid[top10])),
    MAE(y_test, lasso.predict(X_test[top10]))
]
rmse.loc[len(rmse)] = [
    "Lasso shap MinMaxScaler",
    RMSE(y_train, lasso.predict(X_train[top10])),
    RMSE(y_valid, lasso.predict(X_valid[top10])),
    RMSE(y_test, lasso.predict(X_test[top10]))
]
r2.loc[len(r2)] = [
    "Lasso shap MinMaxScaler",
    R2(y_train, lasso.predict(X_train[top10])),
    R2(y_valid, lasso.predict(X_valid[top10])),
    R2(y_test, lasso.predict(X_test[top10]))
]

print("MAE")
display(mae)
print("RMSE")
display(rmse)
print("R2")
display(r2)

,shap_value
bathrooms,525.023919
bedrooms,428.566842
Doorman,308.232523
LaundryInUnit,143.699638
LaundryInBuilding,123.043318
Elevator,94.485483
FitnessCenter,90.345345
NoFee,84.548847
HardwoodFloors,62.929129
Dishwasher,59.074394


MAE


,model,train,valid,test
0,Lasso MinMaxScaler,716.722100,716.634879,711.028730
1,Lasso top10 MinMaxScaler,719.317161,719.755868,714.768131
2,Lasso permutation MinMaxScaler,719.503004,719.809385,714.456090
3,Lasso shap MinMaxScaler,720.222888,718.605434,714.720977


RMSE


,model,train,valid,test
0,Lasso MinMaxScaler,1039.645459,1045.209477,1028.465037
1,Lasso top10 MinMaxScaler,1043.106897,1048.647653,1032.016266
2,Lasso permutation MinMaxScaler,1045.668346,1051.165187,1032.324159
3,Lasso shap MinMaxScaler,1043.769085,1049.054299,1032.413807


R2


,model,train,valid,test
0,Lasso MinMaxScaler,0.577598,0.569756,0.583895
1,Lasso top10 MinMaxScaler,0.574780,0.566921,0.581017
2,Lasso permutation MinMaxScaler,0.572689,0.564839,0.580767
3,Lasso shap MinMaxScaler,0.574240,0.566585,0.580694


### 7. **Hyperparameter optimization**

In [23]:
import numpy as np

from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


#### a. Implement grid search and random search methods for alpha and l1_ratio for sklearn's ElasticNet model.

In [24]:
# CV схема (можно менять)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ElasticNet(max_iter=10000, random_state=42))
])

param_grid = {
    "model__alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0],
    "model__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
}

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1
)

grid.fit(X_train, y_train)

# Random Search (минимально: распределения)
param_dist = {
    "model__alpha": np.logspace(-4, 1, 200),     # 1e-4 ... 10
    "model__l1_ratio": np.linspace(0.05, 0.95, 200)
}

rnd = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=40,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1,
    random_state=42
)

rnd.fit(X_train, y_train)


,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'model__alpha': array([1.0000...00000000e+01]), 'model__l1_ratio': array([0.05 ..., 0.95 ])}"
,n_iter,40
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


#### b. Find the best combination of model hyperparameters.

In [25]:
print("GRID best params:", grid.best_params_)
print("GRID best RMSE:", -grid.best_score_)

print("RND best params:", rnd.best_params_)
print("RND best RMSE:", -rnd.best_score_)


GRID best params: {'model__alpha': 0.01, 'model__l1_ratio': 0.9}
GRID best RMSE: 1039.8628408240934
RND best params: {'model__l1_ratio': np.float64(0.1721105527638191), 'model__alpha': np.float64(0.0021461411978584057)}
RND best RMSE: 1039.862483853019


#### c. Fit the resulting model.

In [26]:
best_search = grid if grid.best_score_ >= rnd.best_score_ else rnd
best_model = best_search.best_estimator_   # Pipeline(scaler + ElasticNet)

best_model.fit(X_test, y_test)


,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,alpha,np.float64(0....1411978584057)
,l1_ratio,np.float64(0.1721105527638191)
,fit_intercept,True
,precompute,False


#### d. Import optuna and configure the same experiment with ElasticNet.

In [27]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.05, 0.95)

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("model", ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000, random_state=42))
    ])
    scores = cross_val_score(
        model, X_train, y_train,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )
    rmse = -scores.mean()
    return rmse


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("OPTUNA best params:", study.best_params)
print("OPTUNA best RMSE:", study.best_value)


[I 2026-01-30 16:08:36,706] A new study created in memory with name: no-name-775411b0-423c-4223-a230-2ec3f642a6ef
[I 2026-01-30 16:08:37,280] Trial 0 finished with value: 1039.863654640581 and parameters: {'alpha': 0.02409705068094126, 'l1_ratio': 0.8646874923702829}. Best is trial 0 with value: 1039.863654640581.
[I 2026-01-30 16:08:37,586] Trial 1 finished with value: 1047.6096039664285 and parameters: {'alpha': 1.4158231086980084, 'l1_ratio': 0.8877488076515275}. Best is trial 0 with value: 1039.863654640581.
[I 2026-01-30 16:08:38,412] Trial 2 finished with value: 1039.8639776575296 and parameters: {'alpha': 0.0002018651211421459, 'l1_ratio': 0.5082583894284903}. Best is trial 0 with value: 1039.863654640581.
[I 2026-01-30 16:08:38,684] Trial 3 finished with value: 1233.9698929228666 and parameters: {'alpha': 2.874018854117548, 'l1_ratio': 0.3095315366069022}. Best is trial 0 with value: 1039.863654640581.
[I 2026-01-30 16:08:39,550] Trial 4 finished with value: 1039.8638163601356 

OPTUNA best params: {'alpha': 0.005176716716559759, 'l1_ratio': 0.6100596895093469}
OPTUNA best RMSE: 1039.862509879244


#### e. Estimate metrics and compare approaches.

In [28]:
from sklearn.model_selection import cross_validate

def cv_metrics(estimator):
    scoring = {
        "rmse": "neg_root_mean_squared_error",
        "mae": "neg_mean_absolute_error",
        "r2": "r2"
    }
    res = cross_validate(estimator, X_valid, y_valid, cv=cv, scoring=scoring, n_jobs=-1)
    return {
        "RMSE": -res["test_rmse"].mean(),
        "MAE": -res["test_mae"].mean(),
        "R2":  res["test_r2"].mean()
    }

grid_metrics = cv_metrics(grid.best_estimator_)
rnd_metrics  = cv_metrics(rnd.best_estimator_)

optuna_best_est = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ElasticNet(**study.best_params, max_iter=10000, random_state=42))
])
opt_metrics = cv_metrics(optuna_best_est)

print("GRID :", grid_metrics)
print("RND  :", rnd_metrics)
print("OPT  :", opt_metrics)


GRID : {'RMSE': np.float64(1047.4072907521982), 'MAE': np.float64(715.6516171753154), 'R2': np.float64(0.5661402221620143)}
RND  : {'RMSE': np.float64(1047.4037854954408), 'MAE': np.float64(715.6198950043121), 'R2': np.float64(0.566143929176107)}
OPT  : {'RMSE': np.float64(1047.4020740648407), 'MAE': np.float64(715.6087812814005), 'R2': np.float64(0.5661456314817859)}


#### f. Run optuna on one of the cross-validation schemes.

In [29]:
from sklearn.model_selection import RepeatedKFold

cv_optuna = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

def objective_rkf(trial):
    alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.05, 0.95)

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("model", ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000, random_state=42))
    ])

    scores = cross_val_score(
        model, X_test, y_test,
        cv=cv_optuna,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )
    return -scores.mean()

study2 = optuna.create_study(direction="minimize")
study2.optimize(objective_rkf, n_trials=50)

print("OPTUNA (RepeatedKFold) best:", study2.best_params, "RMSE:", study2.best_value)


[I 2026-01-30 16:09:13,105] A new study created in memory with name: no-name-eca42d84-be14-4bd8-b1a6-1da9defaeba5
[I 2026-01-30 16:09:13,665] Trial 0 finished with value: 1029.1787314765827 and parameters: {'alpha': 0.009872050760457789, 'l1_ratio': 0.10116571036474858}. Best is trial 0 with value: 1029.1787314765827.
[I 2026-01-30 16:09:14,202] Trial 1 finished with value: 1029.1953906969159 and parameters: {'alpha': 0.0002091408298587435, 'l1_ratio': 0.5162138593555504}. Best is trial 0 with value: 1029.1787314765827.
[I 2026-01-30 16:09:14,587] Trial 2 finished with value: 1029.3844774994354 and parameters: {'alpha': 0.07425917221261999, 'l1_ratio': 0.60171158290786}. Best is trial 0 with value: 1029.1787314765827.
[I 2026-01-30 16:09:14,962] Trial 3 finished with value: 1029.321873018668 and parameters: {'alpha': 0.09399515590874227, 'l1_ratio': 0.7276689814087952}. Best is trial 0 with value: 1029.1787314765827.
[I 2026-01-30 16:09:15,490] Trial 4 finished with value: 1029.1955452

OPTUNA (RepeatedKFold) best: {'alpha': 0.009007653423639562, 'l1_ratio': 0.34086409133123635} RMSE: 1029.1769344020918
